In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, Markdown, Math

seed = 42

_Частина 3_

Завдання: розробити програмне забезпечення для реалізації двошарового персептрону із структурою 2-3-1.

Передбачити режим навчання «ON-LINE» та режим розпізнавання.

Піддослідна функція $х_1+х_2=у$

In [87]:
class NeuralNetwork:
    """
    N-layer neural network mathematic model with sigmoid activation function
    """

    def __init__(self, start_vector, result, npl, add_bias=True):
        """
        Init starting values, first step
        
        Args:
            start_vector: input vector
            result: predicted result after training
            npl: Neuron-per-Layer (i.e. 2-3-1 == [2, 3, 1])
        """
        self._npl = npl
        self.add_bias = add_bias
        # First_neuron_number . depends on bias
        self.one_or_zero = 1 if add_bias else 0
        # Add bias neuron if it needed
        self._inputs = np.concatenate(([1], start_vector)) if add_bias else np.array(start_vector)
        self._result = np.array(result)
        self.weights = []
        for i in range(len(npl)-1):
            w = np.random.default_rng(seed).uniform(
                low=0.3,
                high=0.6,
                size=(npl[i]+self.one_or_zero)*npl[i+1],
            ).round(3).reshape((npl[i]+self.one_or_zero, npl[i+1]))
            self.weights.append(w)
        print("weights", self.weights)
        self.neurons = []
        self.neurons.append(self._inputs)
        for i in range(1, len(npl)):
            self.neurons.append(
                np.concatenate(([1], np.empty(npl[i])))
                if self.add_bias else
                np.empty(npl[i])
            )
        self.epoch_history = []  # same as y_i # but this time it will be in each neuron
        self.error_history = []  # same as dn_i
        self._step_forward()
        

    def sigm(self, x):
        """
        Sigmoidal activation function
        """
        return 1 / (1 + np.exp(-x))

    def sigm_der(self, w):
        """
        Args:
            w: the previous output value of the neuron
        """
        return w * (1 - w)

    def _step_forward(self):
        """
        Make one calculation forward through all NN
        """
        for i in range(1, len(self._npl)):
            print("self.neurons[i-1]", self.neurons[i-1])
            self.neurons[i][self.one_or_zero:] = self.sigm(self.neurons[i-1] @ self.weights[i-1])
            
        self.epoch_history.append(self.neurons[-1][self.one_or_zero:])
        self.error_history.append(
            np.abs((self._result - self.epoch_history[-1]) / self._result)
        )

    def _back_propagate(self, learning_rate):
        """
        Correct previous iteration
        """
        l = learning_rate
        last_delta_w = (self.neurons[-1][self.one_or_zero:] - self._result) * self.sigm_der(self.neurons[-1][self.one_or_zero:])
        
        for i in range(len(self._npl) - 2, -1, -1):
            correction = np.transpose([self.neurons[i]]) @ ([last_delta_w]) * l            
            self.weights[i] -= correction
            last_delta_w = (
                (last_delta_w @ self.weights[i].T)
                * self.sigm_der(self.neurons[i])
            )[self.one_or_zero:]

    def train(self, acceptable_error=0.01, learning_rate=0.1, max_epoch_iteration=50000):
        """
        Train function
        Make forward step and correct weights.
        Loop that untill error will be small enough.
        Limit maximum number of epochs.
        """
        i = 1
        while (self.error_history[-1] > acceptable_error) and i < max_epoch_iteration:
            self._back_propagate(learning_rate)
            self._step_forward()
            print("error", self.error_history[-1])
            i += 1
    
    def predict(self, input_vector):
        """
        Predict input vector with current weights
        """
        return self.sigm(np.dot(input_vector, self.weights))

In [88]:
perceptron = NeuralNetwork([3, 2], [4], [2, 3, 1], add_bias=False)
perceptron.train()

weights [array([[0.532, 0.432, 0.558],
       [0.509, 0.328, 0.593]]), array([[0.532],
       [0.432],
       [0.558]])]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.93175718 0.87566456 0.9458333 ]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.93328423 0.87933801 0.94687178]
error [0.79438023]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.93472519 0.88280572 0.94785498]
error [0.79012734]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.93607705 0.88605353 0.94878065]
error [0.78649645]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.93734144 0.88908191 0.94964954]
error [0.78337941]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.93852258 0.89189964 0.95046414]
error [0.78068738]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9396259  0.89451972 0.95122775]
error [0.77834805]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.94065729 0.89695692 0.95194403]
error [0.77630281]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.94162265 0.89922631 0.95261667]
error [0.7745041]
self.neurons[i-1] [3 2]
self.neuron

error [0.75112821]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97171299 0.96105407 0.97500998]
error [0.75112376]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97173798 0.96109692 0.97503   ]
error [0.75111935]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97176284 0.96113955 0.97504993]
error [0.75111498]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97178758 0.96118195 0.97506976]
error [0.75111063]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9718122  0.96122413 0.97508949]
error [0.75110633]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97183669 0.96126609 0.97510913]
error [0.75110205]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97186107 0.96130783 0.97512868]
error [0.75109781]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97188533 0.96134935 0.97514814]
error [0.75109359]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97190948 0.96139066 0.9751675 ]
error [0.75108942]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97193351 0.96143176 0.97518678]
error [0.75108527]
self.neurons[i-1] [3 

self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97589172 0.96801275 0.97839937]
error [0.75054403]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97590172 0.9680289  0.97840758]
error [0.75054299]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9759117  0.96804501 0.97841578]
error [0.75054194]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97592165 0.96806108 0.97842395]
error [0.75054091]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97593158 0.96807711 0.97843211]
error [0.75053987]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97594149 0.9680931  0.97844025]
error [0.75053884]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97595137 0.96810905 0.97844836]
error [0.75053782]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97596122 0.96812495 0.97845646]
error [0.7505368]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97597106 0.96814082 0.97846454]
error [0.75053578]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97598087 0.96815664 0.9784726 ]
error [0.75053477]
self.neurons[i-1] [3 2]
self.neurons[i-1]

error [0.7503619]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97790436 0.97121488 0.9800626 ]
error [0.75036143]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97791032 0.97122422 0.98006756]
error [0.75036097]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97791628 0.97123355 0.98007251]
error [0.75036051]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97792222 0.97124286 0.98007746]
error [0.75036005]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97792815 0.97125216 0.98008239]
error [0.75035959]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97793408 0.97126144 0.98008732]
error [0.75035913]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97793999 0.9712707  0.98009224]
error [0.75035867]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9779459  0.97127995 0.98009715]
error [0.75035821]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97795179 0.97128918 0.98010205]
error [0.75035776]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97795768 0.9712984  0.98010695]
error [0.7503573]
self.neurons[i-1] [3 2]

error [0.75027226]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9791767  0.97318918 0.98112494]
error [0.750272]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97918086 0.97319557 0.98112843]
error [0.75027174]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97918502 0.97320196 0.98113192]
error [0.75027148]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97918917 0.97320834 0.9811354 ]
error [0.75027121]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97919332 0.97321471 0.98113888]
error [0.75027095]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97919746 0.97322107 0.98114235]
error [0.75027069]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9792016  0.97322743 0.98114582]
error [0.75027043]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97920573 0.97323377 0.98114928]
error [0.75027017]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97920985 0.97324011 0.98115274]
error [0.75026991]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.97921398 0.97324644 0.9811562 ]
error [0.75026965]
self.neurons[i-1] [3 2]

...

error [0.75004464]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98535776 0.98223209 0.9864148 ]
error [0.75004463]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98535819 0.98223269 0.98641517]
error [0.75004463]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98535862 0.98223329 0.98641555]
error [0.75004462]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98535905 0.98223388 0.98641592]
error [0.75004461]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98535948 0.98223448 0.9864163 ]
error [0.75004461]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9853599  0.98223507 0.98641667]
error [0.7500446]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98536033 0.98223567 0.98641704]
error [0.75004459]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98536076 0.98223626 0.98641742]
error [0.75004458]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98536119 0.98223686 0.98641779]
error [0.75004458]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98536162 0.98223745 0.98641817]
error [0.75004457]
self.neurons[i-1] [3 2

error [0.75004283]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98546846 0.98238579 0.9865116 ]
error [0.75004282]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98546886 0.98238636 0.98651196]
error [0.75004281]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98546927 0.98238692 0.98651231]
error [0.75004281]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98546968 0.98238749 0.98651267]
error [0.7500428]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98547008 0.98238805 0.98651303]
error [0.75004279]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98547049 0.98238862 0.98651338]
error [0.75004279]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9854709  0.98238918 0.98651374]
error [0.75004278]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9854713  0.98238974 0.98651409]
error [0.75004278]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98547171 0.98239031 0.98651445]
error [0.75004277]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98547212 0.98239087 0.9865148 ]
error [0.75004276]
self.neurons[i-1] [3 2

self.neurons[i-1] [0.98557212 0.98252947 0.98660233]
error [0.75004117]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98557251 0.98253001 0.98660267]
error [0.75004117]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9855729  0.98253054 0.98660301]
error [0.75004116]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98557329 0.98253108 0.98660335]
error [0.75004115]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98557367 0.98253161 0.98660369]
error [0.75004115]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98557406 0.98253215 0.98660402]
error [0.75004114]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98557445 0.98253269 0.98660436]
error [0.75004114]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98557483 0.98253322 0.9866047 ]
error [0.75004113]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98557522 0.98253376 0.98660504]
error [0.75004112]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98557561 0.98253429 0.98660538]
error [0.75004112]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.985576   0.98253483

error [0.75003965]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98567092 0.98266617 0.98668886]
error [0.75003965]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98567129 0.98266668 0.98668918]
error [0.75003964]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98567166 0.98266719 0.98668951]
error [0.75003964]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98567203 0.9826677  0.98668983]
error [0.75003963]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.9856724  0.98266821 0.98669016]
error [0.75003962]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98567277 0.98266872 0.98669048]
error [0.75003962]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98567314 0.98266923 0.9866908 ]
error [0.75003961]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98567351 0.98266974 0.98669113]
error [0.75003961]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98567388 0.98267025 0.98669145]
error [0.7500396]
self.neurons[i-1] [3 2]
self.neurons[i-1] [0.98567425 0.98267076 0.98669177]
error [0.7500396]
self.neurons[i-1] [3 2]

KeyboardInterrupt: 